<a href="https://colab.research.google.com/github/tomonari-masada/course2025-nlp/blob/main/09_evaluating_answer_choice_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMを使った多肢選択式質問への応答

* 今回は、とりあえず、生成モデルをちょっと使ってみる。

* 以下の記事を参考にした。
  * https://magazine.sebastianraschka.com/p/llm-evaluation-4-approaches

* ランタイムのタイプをGPUに変更しておこう。

In [ ]:
import torch
torch.set_grad_enabled(False)

## データセット

* MMLUデータセットから、`high_school_mathematics`の部分を使う。
  * https://huggingface.co/datasets/cais/mmlu

In [ ]:
from datasets import load_dataset

ds = load_dataset("cais/mmlu", "high_school_mathematics")

In [ ]:
ds

* 今回使用するのは`test`スライス
  * 生成モデルをいきなりそのまま使うだけなので。

## プロンプト作成のためのヘルパ関数

In [ ]:
def format_prompt(example):
    return (
        f"{example['question']}\n"
        f"A. {example['choices'][0]}\n"
        f"B. {example['choices'][1]}\n"
        f"C. {example['choices'][2]}\n"
        f"D. {example['choices'][3]}\n"
        "Answer: "
    )

* プロンプト作成を試してみる。

In [ ]:
ds["test"][0]

* 多肢選択問題になっている。

In [ ]:
print(format_prompt(ds["test"][0]))

## モデルの取得

* 今回はQwenの軽量なLLMにしておく。
  * https://huggingface.co/Qwen/Qwen3-0.6B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, dtype="auto", device_map="auto")

* トークナイザを試してみる。
  * トークナイザの`apply_chat_template`は、あえて使っていない。

In [ ]:
prompt = format_prompt(ds["test"][0])
model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
model_inputs

* どのようなサブワードに分割されているかを見てみる。

In [ ]:
print(tokenizer.convert_ids_to_tokens(model_inputs["input_ids"][0]))

## 答えの生成

In [ ]:
out = model.generate(**model_inputs, max_new_tokens=8, temperature=0.0, do_sample=False)
print(tokenizer.decode(out.squeeze(0)))

In [ ]:
answer = tokenizer.decode(out.squeeze(0)[len(model_inputs["input_ids"][0]):], skip_special_tokens=True)
print(answer)

* 素朴に答えの記号が入っているかどうかのチェックだけしている。

In [ ]:
pred = None
for letter in answer:
    letter = letter.upper()
    if letter in "ABCD":
        pred = letter
        break
if pred is None:
    pred = "N/A"

ans = ds["test"][0]["answer"]
gold = "ABCD"[ans] if isinstance(ans, int) else str(ans).strip().upper()

print(f"Predicted: {pred}, Correct: {gold}")

* ここまでの処理をヘルパ関数としてまとめる。

In [ ]:
def predict_choice(example):
    prompt = format_prompt(example)

    model_inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    out = model.generate(**model_inputs, max_new_tokens=8, temperature=0.0, do_sample=False)
    answer = tokenizer.decode(out.squeeze(0)[len(model_inputs["input_ids"][0]):], skip_special_tokens=True)

    pred = None
    for letter in answer:
        letter = letter.upper()
        if letter in "ABCD":
            pred = letter
            break
    if pred is None:
        pred = "N/A"

    ans = example["answer"]
    gold = "ABCD"[ans] if isinstance(ans, int) else str(ans).strip().upper()

    return pred, gold

In [ ]:
predict_choice(ds["test"][0])

## 評価

* どのくらい正答するか、少し様子を見てみる。

In [ ]:
for i in range(10):
    pred, gold = predict_choice(ds["test"][i])
    print(f"Q{i+1}: Predicted: {pred}, Correct: {gold}")

* `test`スライス全体で正解率を求める。

In [ ]:
import time

total = 0
correct = 0
start = time.time()
for i in range(len(ds["test"])):
    pred, gold = predict_choice(ds["test"][i])
    total += 1
    if pred == gold:
        correct += 1
    if (i + 1) % 10 == 0:
        end = time.time()
        print(f"Processed {i+1}/{len(ds['test'])} in {end - start:.1f} sec")
        start = end
        print(f"  Current accuracy: {correct}/{total} = {correct/total:.3%}")
end = time.time()
print(f"Accuracy: {correct}/{total} = {correct/total:.3%}")

In [ ]:
def format_prompt(example):
    QA_test = (
         "You are a highly intelligent question answering bot. "
         "If you don't know the answer, just say 'N/A'. "
         "Do not attempt to fabricate an answer.\n\n"
    )
    prompt = (
        QA_test +
        f"{example['question']}\n"
        f"A. {example['choices'][0]}\n"
        f"B. {example['choices'][1]}\n"
        f"C. {example['choices'][2]}\n"
        f"D. {example['choices'][3]}\n"
        "Answer: "
    )
    messages = [
        {"role": "system", "content": prompt}
    ]
    return messages

In [ ]:
messages = format_prompt(ds["test"][0])

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]

In [ ]:
answer = tokenizer.decode(output_ids, skip_special_tokens=True)
print(answer)